# Train the Grader

In [1]:
import torch.utils.data as Data
import torch

data = torch.load(
    "Data/Beat_dataset/Beat_tensor_dataset/beat_no_head_rsz_75.pt")
x = data['x']
y = data['y']
batch_size = 64
dataset = Data.TensorDataset(x, y)
train_iter = Data.DataLoader(dataset, batch_size, shuffle=True, num_workers=0)

In [2]:
from GenMusic.models.mlp import MLP

mlp = MLP(75, [150, 50, 10], 2)

In [3]:
from torch import nn

lr = 1e-3
epoch = 1000
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(mlp.parameters(), lr=lr)
device = torch.device("cuda")
torch.cuda.empty_cache()  # 释放显存

In [ ]:
from GenMusic.models import train

model = train(train_iter, mlp, criterion,
              optimizer, epoch, device, autoPrint=0)

# Generate beat

In [1]:
import torch
from GenMusic.bin.rewarders import nngrader_rewarder, get_nngrader_rewarder
from GenMusic import ga_generator

rwd = get_nngrader_rewarder("runs/1708865989.470388/best.pt")
rwd.set_tgt(torch.tensor([0.5, 0.5]))
ga = ga_generator(rwd)

In [2]:
from configs import *
mxomin = [1]
dim = INPUT_NN_LEN
varTypes = [0]*INPUT_NN_LEN
lb = [0.3]*INPUT_NN_LEN
ub = [0.9]*INPUT_NN_LEN
lbin = [1]*INPUT_NN_LEN
ubin = [1]*INPUT_NN_LEN
ga.set_problem_params(maxormins=mxomin, Dim=dim,
                      varTypes=varTypes, lb=lb, ub=ub, lbin=lbin, ubin=lbin)
ga.set_problem_aimFunc(ga.aimFun_1)
ga.set_population_params()
ga.set_algorithm_params()

In [3]:
ga.generate_ori()

In [4]:
print(ga.best_tensor)

tensor([0.6442, 0.4159, 0.7559, 0.8330, 0.8718, 0.5822, 0.6580, 0.5149, 0.3594,
        0.5778, 0.7941, 0.8634, 0.5797, 0.6006, 0.4633, 0.3060, 0.4502, 0.7285,
        0.7136, 0.4186, 0.6839, 0.8184, 0.8537, 0.5099, 0.3503, 0.7018, 0.5898,
        0.8379, 0.6241, 0.7451, 0.6107, 0.8377, 0.5890, 0.7408, 0.7844, 0.7361,
        0.8662, 0.5173, 0.5500, 0.6087, 0.8681, 0.4814, 0.8755, 0.5047, 0.6441,
        0.5195, 0.5944, 0.4067, 0.4248, 0.8595, 0.6523, 0.8596, 0.7491, 0.4129,
        0.7651, 0.4880, 0.5711, 0.5494, 0.7821, 0.3679, 0.7126, 0.4755, 0.6046,
        0.6944, 0.6085, 0.8312, 0.7805, 0.6609, 0.5054, 0.8087, 0.6968, 0.7450,
        0.4895, 0.5293, 0.7676])


# midi GM

In [1]:
import torch
from GenMusic.bin.rewarders import nngrader_rewarder, get_nngrader_rewarder, sad_test_rewarder
from GenMusic import ga_generator

rwd = sad_test_rewarder()
ga = ga_generator(rwd)

In [5]:
from configs import *
mxomin = [-1]
dim = INPUT_NN_LEN
varTypes = [1]*INPUT_NN_LEN
lb = [0]*INPUT_NN_LEN
ub = [11]*(10)
ub += [3]*(10)
lbin = [1]*INPUT_NN_LEN
ubin = [1]*INPUT_NN_LEN
ga.set_problem_params(maxormins=mxomin, Dim=dim,
                      varTypes=varTypes, lb=lb, ub=ub, lbin=lbin, ubin=lbin)
ga.set_problem_aimFunc(ga.aimFun_2)
ga.set_population_params()
ga.set_algorithm_params()

ga.generate_ori()

In [6]:
print(ga.best_tensor)

tensor([0., 0., 4., 9., 9., 4., 4., 4., 4., 2., 1., 0., 1., 1., 1., 0., 1., 0.,
        0., 0.])


In [26]:
import mido

# 加载 MIDI 文件
mid = mido.MidiFile('/home/csy/Desktop/GenMusic/happy_melody.mid')
c1=0
c2=0
# 遍历所有的轨道和消息
for track in mid.tracks:
    for msg in track:
        if msg.type== 'note_on':
            c1+=1
        if msg.type== 'note_off':
            c2+=1       
        print(msg)
print(c1)
print(c2)


MetaMessage('set_tempo', tempo=500000, time=0)
MetaMessage('end_of_track', time=0)
MetaMessage('track_name', name='Happy Melody', time=0)
program_change channel=0 program=0 time=0
note_on channel=0 note=9 velocity=100 time=0
note_on channel=0 note=5 velocity=100 time=960
note_off channel=0 note=9 velocity=100 time=480
note_on channel=0 note=9 velocity=100 time=0
note_off channel=0 note=9 velocity=100 time=960
note_on channel=0 note=9 velocity=100 time=0
note_off channel=0 note=5 velocity=100 time=480
note_off channel=0 note=9 velocity=100 time=1440
note_on channel=0 note=9 velocity=100 time=0
note_on channel=0 note=2 velocity=100 time=3840
note_on channel=0 note=5 velocity=100 time=480
note_off channel=0 note=2 velocity=100 time=1440
note_off channel=0 note=5 velocity=100 time=480
note_on channel=0 note=5 velocity=100 time=0
note_off channel=0 note=5 velocity=100 time=5760
note_off channel=0 note=9 velocity=100 time=3360
MetaMessage('end_of_track', time=0)
8
8


In [17]:
import random
from midiutil import MIDIFile

# 定义可能的节奏和音高范围
rhythms = ['全音符', '二分音符', '四分音符', '八分音符']
pitches = ['C3', 'C#3', 'D3', 'D#3', 'E3',
           'F3', 'F#3', 'G3', 'G#3', 'A3', 'A#3', 'B3']

# 将音符名称转换为MIDI音符编号


def note_name_to_midi(note_name):
    midi_notes = {
        'C3': 0, 'C#3': 1, 'D3': 2, 'D#3': 3, 'E3': 4,
        'F3': 5, 'F#3': 6, 'G3': 7, 'G#3': 8, 'A3': 9, 'A#3': 10, 'B3': 11
    }
    return midi_notes[note_name]

# 将节奏转换为MIDI持续时间（以四分音符为单位）


def rhythm_to_midi(rhythm):
    if rhythm == '全音符':
        return 4
    elif rhythm == '二分音符':
        return 2
    elif rhythm == '四分音符':
        return 1
    elif rhythm == '八分音符':
        return 0.5
    else:
        return 1

# 评分函数，评估乐谱的快乐程度

# 创建MIDI文件


def create_midi(melody_rhythm, melody_pitches):
    midi_file = MIDIFile(1)  # 创建一个包含1个轨道的MIDI文件
    track = 0
    time = 0

    # 设置钢琴音色
    midi_file.addProgramChange(track, time, 0, 0)  # 音色编号0对应钢琴

    midi_file.addTrackName(track, time, "Happy Melody")
    midi_file.addTempo(track, time, 120)  # 设置BPM为120

    for note_name, rhythm in zip(melody_pitches, melody_rhythm):
        midi_note = note_name_to_midi(note_name)
        duration = rhythm_to_midi(rhythm)
        midi_file.addNote(track, 0, midi_note, time, int(duration * 4), 100)
        time += duration

    with open("happy_melody.mid", 'wb') as output_file:
        midi_file.writeFile(output_file)


create_midi(['八分音符','二分音符'],['C3','C#3'])

In [19]:
from midiutil import MIDIFile

# 创建一个包含1个轨道的MIDI文件
midi_file = MIDIFile(1)

track = 0   # 轨道编号
time = 0    # 开始时间

# 设置轨道名和节拍
midi_file.addTrackName(track, time, "C Major Scale")
midi_file.addTempo(track, time, 120)

# 定义音符（C4-E4-G4-C5）和持续时间（都是四分音符）
notes = [60, 64, 67, 72]  # MIDI音符编号
durations = [1, 1, 1, 1]  # 持续时间，以四分音符为单位

# 添加音符到轨道
for note, duration in zip(notes, durations):
    midi_file.addNote(track, 0, note, time, duration, 100)
    time += duration

# 写入文件
with open("CMajorScale.mid", 'wb') as output_file:
    midi_file.writeFile(output_file)


In [27]:
import mido

# MIDI音符编号到音名的映射
note_names = {
    0: 'C3', 1: 'C#3', 2: 'D3', 3: 'D#3', 4: 'E3',
    5: 'F3', 6: 'F#3', 7: 'G3', 8: 'G#3', 9: 'A3', 10: 'A#3', 11: 'B3'
}

# 将MIDI音符编号转换为音名
def midi_to_note_name(midi_note):
    octave = midi_note // 12 - 1
    note = midi_note % 12
    return note_names[note] + str(octave)

# 将MIDI持续时间（以四分音符为单位）转换为节奏
def midi_to_rhythm(midi_duration, ticks_per_beat):
    if midi_duration == ticks_per_beat * 4:
        return '全音符'
    elif midi_duration == ticks_per_beat * 2:
        return '二分音符'
    elif midi_duration == ticks_per_beat:
        return '四分音符'
    elif midi_duration == ticks_per_beat / 2:
        return '八分音符'
    else:
        return '未知音符'

def midi_to_notes_rhythm(midi_file_path):
    mid = mido.MidiFile(midi_file_path)
    ticks_per_beat = mid.ticks_per_beat

    notes = []
    rhythm = []

    for track in mid.tracks:
        time = 0
        for msg in track:
            if msg.type == 'note_on' and msg.velocity > 0:
                note_name = midi_to_note_name(msg.note)
                notes.append(note_name)
            elif msg.type == 'note_off' or (msg.type == 'note_on' and msg.velocity == 0):
                rhythm_name = midi_to_rhythm(msg.time, ticks_per_beat)
                rhythm.append(rhythm_name)
                time = 0
            else:
                time += msg.time

    return notes, rhythm

notes, rhythm = midi_to_notes_rhythm('/home/csy/Desktop/GenMusic/Data/mini_midi/happy/happy1.mid')
print('Notes:', notes)
print('Rhythm:', rhythm)


Notes: ['G34', 'G#34', 'A#34', 'A#34', 'A#34', 'A#34', 'A#34', 'A#34', 'A#34', 'C35', 'G34', 'G#34', 'A#34', 'A#34', 'A#34', 'G#34', 'G#34', 'G#34', 'G#34', 'G#34', 'G#34', 'G#34', 'G34', 'F34', 'G34', 'G#34']
Rhythm: ['未知音符', '未知音符', '未知音符', '未知音符', '未知音符', '未知音符', '未知音符', '未知音符', '未知音符', '未知音符', '未知音符', '未知音符', '未知音符', '未知音符', '未知音符', '未知音符', '未知音符', '未知音符', '未知音符', '未知音符', '未知音符', '未知音符', '未知音符', '未知音符', '未知音符', '未知音符']
